# Mapping PlayerIds from FanGraphs and StatCast
Using the data from [Chadwick Bureau](https://github.com/chadwickbureau), map the player IDs from FanGraphs to their IDs on StatCast (which uses MLBAM IDs)

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [2]:
# Read data from Chadwick Bureau
people_0 = pd.read_csv("data/chadwick-bureau/people-0.csv")
people_1 = pd.read_csv("data/chadwick-bureau/people-1.csv")
people_2 = pd.read_csv("data/chadwick-bureau/people-2.csv")
people_3 = pd.read_csv("data/chadwick-bureau/people-3.csv")
people_4 = pd.read_csv("data/chadwick-bureau/people-4.csv")
people_5 = pd.read_csv("data/chadwick-bureau/people-5.csv")
people_6 = pd.read_csv("data/chadwick-bureau/people-6.csv")
people_7 = pd.read_csv("data/chadwick-bureau/people-7.csv")
people_8 = pd.read_csv("data/chadwick-bureau/people-8.csv")
people_9 = pd.read_csv("data/chadwick-bureau/people-9.csv")
people_a = pd.read_csv("data/chadwick-bureau/people-a.csv")
people_b = pd.read_csv("data/chadwick-bureau/people-b.csv")
people_c = pd.read_csv("data/chadwick-bureau/people-c.csv")
people_d = pd.read_csv("data/chadwick-bureau/people-d.csv")
people_e = pd.read_csv("data/chadwick-bureau/people-e.csv")
people_f = pd.read_csv("data/chadwick-bureau/people-f.csv")

# Concatenate data
id_mapping_df = pd.concat(
    [
        people_0,
        people_1,
        people_2,
        people_3,
        people_4,
        people_5,
        people_6,
        people_7,
        people_8,
        people_9,
        people_a,
        people_b,
        people_c,
        people_d,
        people_e,
        people_f,
    ],
    axis=0,
)

# Remove unnecessary columns
id_mapping_df = id_mapping_df[
    [
        "key_person",
        "key_uuid",
        "key_mlbam",
        "key_retro",
        "key_bbref",
        "key_bbref_minors",
        "key_fangraphs",
        "name_last",
        "name_first",
        "name_given",
        "pro_played_first",
        "pro_played_last",
    ]
]

# Remove any players without a FanGraphs/StatCast id
id_mapping_df = id_mapping_df[id_mapping_df["key_fangraphs"].notna()]
id_mapping_df = id_mapping_df[id_mapping_df["key_mlbam"].notna()]
id_mapping_df = id_mapping_df[id_mapping_df["pro_played_first"].notna()]
id_mapping_df = id_mapping_df[id_mapping_df["pro_played_last"].notna()]


# Change column type from float to int
id_mapping_df["key_mlbam"] = id_mapping_df["key_mlbam"].astype(int)
id_mapping_df["key_fangraphs"] = id_mapping_df["key_fangraphs"].astype(int)
id_mapping_df["pro_played_first"] = id_mapping_df["pro_played_first"].astype(int)
id_mapping_df["pro_played_last"] = id_mapping_df["pro_played_last"].astype(int)

id_mapping_df

,key_person,key_uuid,key_mlbam,key_retro,key_bbref,key_bbref_minors,key_fangraphs,name_last,name_first,name_given,pro_played_first,pro_played_last
49,000539fc,000539fc-40b1-4bc4-9764-2941d18f398c,605152,bradj002,bradlje01,bradle000jed,13166,Bradley,Jed,Jedidiah Custer,2011,2019
56,000638f5,000638f5-74e9-4e4f-99d2-47969ae2d7a8,110625,barrm002,barrima01,barrio001man,1000605,Barrios,Manuel,Manuel Antonio,1994,2003
59,0007057d,0007057d-9e3c-4db2-9ceb-989cd788605e,118336,martf102,martifr01,martin001fra,1008165,Martin,Frank,Frank Joseph,1897,1905
69,00087a47,00087a47-2151-4fcd-8952-98e915e88143,111603,browb101,brownby01,browne002byr,1001500,Browne,Byron,Byron Ellis,1963,1975
122,00109852,00109852-1ac0-4751-b0ed-11ab819d8ba3,113051,darir101,darinro01,daring001rol,1002986,Daringer,Rolla,Rolla Harrison,1910,1921
...,...,...,...,...,...,...,...,...,...,...,...,...
32189,ffef3420,ffef3420-0132-4e68-86c9-615fd859e9e8,120175,pagev101,pageva01,page--001van,1009957,Page,Vance,Vance Linwood,1926,1942
32235,fff5491d,fff5491d-9cb9-48af-a4b6-67796f7f3ceb,124274,wilkr101,wilkiro01,wilkin003roy,1013961,Wilkinson,Roy,Roy Hamilton,1913,1932
32261,fff74cf7,fff74cf7-a005-4a4a-b361-95b13908b65d,113250,demaa101,demaral01,demare001alb,1003187,Demaree,Al,Albert Wentworth,1908,1924
32290,fffb2763,fffb2763-f758-4d94-af65-ee60117fea4d,118767,mckef101,mckeefr01,mckee-001fra,1008583,McKee,Frank,Frank,1884,1884


## Create ID mapping

In [3]:
mlb_stats_df = pd.read_csv("data/full-mlb-stats.csv")
mlb_stats_df = mlb_stats_df[["Name", "StatCast_ID"]]

final_id_mapping_df = mlb_stats_df.merge(
    id_mapping_df, how="inner", left_on="StatCast_ID", right_on="key_mlbam"
)

final_id_mapping_df = final_id_mapping_df[["Name", "StatCast_ID", "key_fangraphs"]]
final_id_mapping_df.rename(columns={"key_fangraphs": "FanGraphs_ID"}, inplace=True)
final_id_mapping_df

,Name,StatCast_ID,FanGraphs_ID
0,A.J. Burnett,150359,512
1,A.J. Cole,595918,11467
2,A.J. Ellis,454560,5677
3,A.J. Ellis,454560,5677
4,A.J. Ellis,454560,5677
...,...,...,...
7125,Óscar Mercado,640458,16375
7126,Óscar Mercado,640458,16375
7127,Óscar Mercado,640458,16375
7128,Óscar Mercado,640458,16375


In [4]:
# Save data
final_id_mapping_df.to_csv("data/final_id_mapping.csv", index=False)

## Mapping players from minors to majors
Conduct an inner join between `full-mlb-stats.csv` with `final_id_mapping_df.csv` on StatCastID. Then, conduct an inner join of the new table with `milb-players.csv` on FanGraphsID.

In [5]:
# Read data
mlb_players = pd.read_csv("data/full-mlb-stats.csv")
mlb_career_stats_df = pd.read_csv("data/mlb-career-stats.csv")
milb_players_df = pd.read_csv("data/milb-players.csv")
final_id_mapping_df = pd.read_csv("data/final_id_mapping.csv")

# Inner join between full-mlb-stats and final_id_mapping_df
mapped_mlb_players = mlb_career_stats_df.merge(
    final_id_mapping_df, how="inner", left_on="StatCast_ID", right_on="StatCast_ID"
).drop_duplicates()
mapped_mlb_players.drop("Name_y", axis=1, inplace=True)
mapped_mlb_players.rename(columns={"Name_x": "Name"}, inplace=True)

# Inner join bettwen mapped_mlb_players and milb-players
full_mapping = mapped_mlb_players.merge(
    milb_players_df, how="inner", left_on="FanGraphs_ID", right_on="PlayerId"
).drop_duplicates()

# Drop columns
full_mapping.drop(["Name_y", "PlayerId"], axis=1, inplace=True)

In [6]:
# Rename columns
full_mapping.rename(
    columns={
        "Name_x": "Name",
        "AB": "major_AB",
        "PA_x": "major_PA",
        "1B": "major_1B",
        "2B": "major_2B",
        "3B": "major_3B",
        "HR": "major_HR+",
        "K": "major_K",
        "BB": "major_BB",
        "K%_x": "major_K%",
        "BB%_x": "major_BB%",
        "BA": "major_BA",
        "SLG_x": "major_SLG",
        "OBP_x": "major_OBP",
        "OPS_x": "major_OPS",
        "ISO_x": "major_ISO",
        "BABIP_x": "major_BABIP",
        "CS": "major_CS",
        "SB": "major_SB",
        "HBP": "major_HBP",
        "IBB": "major_IBB",
        "SF": "major_SF",
        "Speed": "major_Speed",
        "wOBA_x": "major_wOBA",
        "wRAA_x": "major_wRAA",
        "wRC_x": "major_wRC",
        "Team": "minor_Team",
        "Level": "minor_Level",
        "PA_y": "minor_PA",
        "BB%_y": "minor_BB%",
        "K%_y": "minor_K%",
        "BB/K": "minor_BB/K",
        "AVG": "minor_BA",
        "OBP_y": "minor_OBP",
        "SLG_y": "minor_SLG",
        "OPS_y": "minor_OPS",
        "ISO_y": "minor_ISO",
        "Spd": "minor_Speed",
        "BABIP_y": "minor_BABIP",
        "wSB": "minor_wSB",
        "wRC_y": "minor_wRC",
        "wRAA_y": "minor_wRAA",
        "wOBA_y": "minor_wOBA",
        "wRC+": "minor_wRC+",
    },
    inplace=True,
)

In [7]:
# Reorganize columns
full_mapping = full_mapping[
    [
        "Name",
        "Age",
        "StatCast_ID",
        "FanGraphs_ID",
        "Season",
        "minor_Level",
        "minor_Team",
        "minor_PA",
        "minor_BB%",
        "minor_K%",
        "minor_BB/K",
        "minor_BA",
        "minor_OBP",
        "minor_SLG",
        "minor_OPS",
        "minor_ISO",
        "minor_Speed",
        "minor_BABIP",
        "minor_wSB",
        "minor_wRC",
        "minor_wRAA",
        "minor_wOBA",
        "minor_wRC+",
        "major_AB",
        "major_PA",
        "major_1B",
        "major_2B",
        "major_3B",
        "major_HR+",
        "major_K",
        "major_BB",
        "major_K%",
        "major_BB%",
        "major_BA",
        "major_SLG",
        "major_OBP",
        "major_OPS",
        "major_ISO",
        "major_BABIP",
        "major_CS",
        "major_SB",
        "major_HBP",
        "major_IBB",
        "major_SF",
        "major_Speed",
        "major_wOBA",
        "major_wRAA",
        "major_wRC",
    ]
]

# Sort
full_mapping.sort_values(by=["Name", "Age"], inplace=True)
full_mapping.reset_index(drop=True, inplace=True)

# Display
full_mapping

,Name,Age,StatCast_ID,FanGraphs_ID,Season,minor_Level,minor_Team,minor_PA,minor_BB%,minor_K%,...,major_BABIP,major_CS,major_SB,major_HBP,major_IBB,major_SF,major_Speed,major_wOBA,major_wRAA,major_wRC
0,AJ Reed,22,607223,16246,2015,A+,HOU,385,0.153247,0.189610,...,0.232,0,0,0,0,2,23.95,0.229,-14.259215,8.552785
1,AJ Reed,24,607223,16246,2017,AAA,HOU,556,0.129496,0.262590,...,0.232,0,0,0,0,2,23.95,0.229,-14.259215,8.552785
2,AJ Reed,25,607223,16246,2018,AAA,HOU,540,0.118519,0.237037,...,0.232,0,0,0,0,2,23.95,0.229,-14.259215,8.552785
3,Aaron Judge,24,592450,15640,2016,AAA,NYY,410,0.114634,0.239024,...,0.348,21,64,39,99,32,27.21,0.424,445.052420,1036.729420
4,Aaron Whitefield,20,664334,19938,2017,A,MIN,460,0.067391,0.256522,...,0.000,0,0,0,0,0,NaN,0.000,-2.708499,-1.454499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,Óscar Mercado,20,640458,16375,2015,A,STL,513,0.044834,0.118908,...,0.270,8,29,8,0,6,28.58,0.291,-21.241482,98.409518
1844,Óscar Mercado,21,640458,16375,2016,A+,STL,506,0.086957,0.140316,...,0.270,8,29,8,0,6,28.58,0.291,-21.241482,98.409518
1845,Óscar Mercado,22,640458,16375,2017,AA,STL,523,0.061185,0.214149,...,0.270,8,29,8,0,6,28.58,0.291,-21.241482,98.409518
1846,Óscar Mercado,23,640458,16375,2018,AAA,STL,427,0.084309,0.149883,...,0.270,8,29,8,0,6,28.58,0.291,-21.241482,98.409518


In [8]:
# Save data
full_mapping.to_csv("data/minors_to_majors.csv", index=False)